### Generating a json file with contentious terms by ContentiousIssues

In [ ]:
import json
import pandas as pd
import rdflib
from rdflib import Graph
from rdflib.namespace import Namespace
from rdflib.namespace import SKOS, RDF

In [ ]:
culco = Namespace("http://w3id.org/culco#")
skosxl = Namespace("http://www.w3.org/2008/05/skos-xl#")
dcterms = Namespace("http://purl.org/dc/terms/")

In [ ]:
path_to_wm = 'https://github.com/cultural-ai/wordsmatter/raw/main/glossary.ttl'

In [ ]:
wm = Graph()
wm.parse(path_to_wm, format="turtle")

In [ ]:
CI = wm.query(
    """        
        SELECT ?CI ?CI_title ?contentious_label ?contentious_label_lit

        WHERE {

          ?CI a culco:ContentiousIssue ;
                culco:hasContentiousLabel ?contentious_label ;
                dcterms:title ?CI_title .

          ?contentious_label skosxl:literalForm ?contentious_label_lit .

        }
    """,
    
    initNs={'culco': culco, 'skosxl':skosxl, 'dcterms':dcterms}
    
)

In [ ]:
CI_URIs = []
CI_titles = []
CI_lang = []
cont_labels_URIs = []
cont_labels_lit = []
cont_label_lang = []

for row in CI:
    
    CI_URIs.append(row.CI.split('#')[-1])
    CI_lang.append(row.CI_title.language)
    CI_titles.append(row.CI_title.value)
    cont_labels_URIs.append(row.contentious_label.split('#')[-1])
    cont_labels_lit.append(row.contentious_label_lit.value)
    cont_label_lang.append(row.contentious_label_lit.language)

data_df = {"CI_URI":CI_URIs,
           "CI_title":CI_titles,
           "CI_lang":CI_lang,
           "cont_label_URI":cont_labels_URIs,
           "cont_label_lit":cont_labels_lit,
           "cont_label_lang":cont_label_lang}

In [ ]:
CI_df = pd.DataFrame(data_df)

In [ ]:
CI_df

In [ ]:
WM_json = {}

for group in CI_df.groupby('CI_URI'):
        
    CI_dict = {"title":group[1]['CI_title'].values[0],"CI_lang":group[1]['CI_lang'].values[0]}
    
    cont_labels_dict = {}

    cont_labels = list(zip(group[1]['cont_label_URI'].values,group[1]['cont_label_lit'].values,group[1]['cont_label_lang'].values))
    
    for label in cont_labels:
        cont_labels_dict[label[0]] = {"literal_form":label[1], "lang":label[2]}
        
    CI_dict["contentious_labels"] = cont_labels_dict
        
    WM_json[group[1]['CI_URI'].values[0]] = CI_dict

In [ ]:
with open('CI_with_cont_terms.json', 'w') as jf:
    json.dump(WM_json, jf)